# Convolutional neural network

>*You tested above different models with the set of high-level features extracted from a pretrained neural network. However, can you get similar results by (re)training a ConvNet from the pixels?*
>* *What accuracy can you achieve?*
>* *Can you get good results? - If not, why?*

In [1]:
# Import the packages needed 
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
import pandas as pd
import tensorflow.keras as keras

/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/exts-ml/lib/python3.6/site-packages/tensorflow/

I am going to start by loading the images (pixels) and labels from all the sets (train, validation and test sets).

In [2]:
# Load images and labels from npz files
with np.load('train.npz', allow_pickle=False) as npz_file:
    X_train=npz_file['images']
    y_train=npz_file['labels']

with np.load('valid.npz', allow_pickle=False) as npz_file:
    X_valid=npz_file['images']
    y_valid=npz_file['labels']

with np.load('test.npz', allow_pickle=False) as npz_file:
    X_test=npz_file['images']
    y_test=npz_file['labels']

I am going to implement a convolutional neural network with keras api. Let's start by creating the model and adding the layers. The input dimension corresponds to the shape of the pixels for one image. I am also going to print a summary of the network.

In [3]:
# Convolutional Network
model = keras.Sequential()
model.add(keras.layers.Conv2D(filters=64, kernel_size=5, strides=2,
                              activation='relu', input_shape=(224, 224, 3)))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Conv2D(filters=64, kernel_size=3, strides=1,
                              activation='relu'))
model.add(keras.layers.MaxPool2D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=6, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 64)      4864      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 55, 55, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 53, 53, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 26, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 43264)             0         
_________________________________________________________________
dense (Dense)                (None, 6)                 259590    
Total params: 301,382
Trainable params: 301,382
Non-trainable params: 0
_________________________________________________________________


I am going to define the loss function, the optimizer and the metrics that I want to monitor with the compile function.

In [4]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['acc'])

I am also going to add an earlystopping function that will monitor the loss value on the validation set and stop the training if this loss value does not improve anymore for more than 6 epochs.

In [5]:
# End training when accuracy stops improving
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

I am going to fit the network using the train set and setting the validation data to the validation set in order to see the results on the validation set. I do not need to suffle as this was done when I saved the images.

In [6]:
# Train model
history = model.fit(
  x=X_train, y=y_train, validation_data=(X_valid,y_valid),
    batch_size=32, epochs=100, shuffle=False, callbacks=[early_stopping] # No need for shuffling and this was done before training samples
)

Train on 281 samples, validate on 139 samples
Epoch 1/100
281/281 [==============================] - 9s 34ms/step - loss: 1.8438 - acc: 0.1993 - val_loss: 1.7321 - val_acc: 0.2446
Epoch 2/100
281/281 [==============================] - 8s 27ms/step - loss: 1.6558 - acc: 0.3986 - val_loss: 1.6244 - val_acc: 0.4245
Epoch 3/100
281/281 [==============================] - 8s 27ms/step - loss: 1.3708 - acc: 0.6050 - val_loss: 1.5044 - val_acc: 0.3957
Epoch 4/100
281/281 [==============================] - 8s 27ms/step - loss: 0.9941 - acc: 0.7011 - val_loss: 1.4416 - val_acc: 0.4604
Epoch 5/100
281/281 [==============================] - 8s 27ms/step - loss: 0.6548 - acc: 0.8114 - val_loss: 1.4793 - val_acc: 0.4676
Epoch 6/100
281/281 [==============================] - 8s 27ms/step - loss: 0.3734 - acc: 0.8932 - val_loss: 1.6493 - val_acc: 0.4604
Epoch 7/100
281/281 [==============================] - 8s 29ms/step - loss: 0.2093 - acc: 0.9466 - val_loss: 2.1823 - val_acc: 0.3165
Epoch 8/100
281/

I am going to compute the test accuracy and save it in an csv file.

In [7]:
test_loss, test_acc = model.evaluate(X_test,  y_test)

60/60 [==============================] - 1s 10ms/step


In [8]:
test_acc

0.4499999980131785

In [9]:
# save results
results=pd.DataFrame({
        'model': ['cnn'],
        'test_accuracy': '{:.3f}'.format(test_acc)
    })

In [10]:
# add to csv file with results
pd.read_csv('results.csv').append(results).to_csv('results.csv', index=False)

The results of training a ConvNet from the pixels are not as good as using the high-level features extracted from a pretrained neural network. This was expected as it is easier for the models to predict when the high-level features are already identified. 